##TEAM MEMBERS

Aidan Stautland, 502841; Christian Torhaug, 564355; Jens Skaug, STUDID

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import geopandas as gpd
from geopy.distance import geodesic
from shapely.geometry import Point

ais_sample_submission.csv

In [3]:
sampledf = pd.read_csv("ais_sample_submission.csv")

sampledf["ID"] = sampledf["ID"].apply(lambda x: int(x))
sampledf["latitude_predicted"] = sampledf["latitude_predicted"].apply(lambda x: float(x))
sampledf["longitude_predicted"] = sampledf["longitude_predicted"].apply(lambda x: float(x))

ais_test.csv

In [4]:
testdf = pd.read_csv("ais_test.csv")

testdf["time"] = testdf["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
testdf["scaling_factor"] = testdf["scaling_factor"].apply(lambda x: float(x))

ais_train.csv

In [5]:
traindf = pd.read_csv("~/Ais_Data/ais_train.csv", sep="|")

def parseEta(row):
    try:
        return datetime.strptime(row['etaRaw'], "%m-%d %H:%M")
    except ValueError:
        return ""

#traindf["time"] = traindf["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
traindf["time"] = pd.to_datetime(traindf["time"]) 
traindf["cog"] = traindf["cog"].apply(lambda x: float(x) if float(x) < 360.0 else 360.0)
traindf["sog"] = traindf["sog"].apply(lambda x: float(x) if float(x) <= 30.0 else 0)
traindf["rot"] = traindf["rot"].apply(lambda x: int(x))
traindf["heading"] = traindf["heading"].apply(lambda x: int(x) if int(x) < 360.0 else 360.0)
traindf["etaRaw"] = traindf.apply(parseEta, axis=1)
traindf["latitude"] = traindf["latitude"].apply(lambda x: float(x))
traindf["longitude"] = traindf["longitude"].apply(lambda x: float(x))

traindf["navstat"] = traindf["navstat"].apply(lambda x: int(x) if x in [0, 1, 5, 8] else 15)
traindf["navstat"] = traindf["navstat"].replace(8, 0)

ports.csv

In [6]:
portsdf = pd.read_csv("ports.csv", sep="|")

portsdf["latitude"] = portsdf["latitude"].apply(lambda x: float(x))
portsdf["longitude"] = portsdf["longitude"].apply(lambda x: float(x))

schedules_to_may_2024.csv

In [7]:
schedulesdf = pd.read_csv("~/Ais_Data/schedules_to_may_2024.csv", sep="|")
schedulesdf = schedulesdf.drop_duplicates()
schedulesdf = schedulesdf.dropna(subset=['portName'])

schedulesdf["arrivalDate"] = pd.to_datetime(schedulesdf["arrivalDate"], errors='coerce')
schedulesdf["arrivalDate"] = schedulesdf["arrivalDate"].dt.tz_convert('UTC').dt.tz_localize(None)
schedulesdf["sailingDate"] = pd.to_datetime(schedulesdf["sailingDate"], errors='coerce')
schedulesdf["sailingDate"] = schedulesdf["sailingDate"].dt.tz_convert('UTC').dt.tz_localize(None)
schedulesdf["portLatitude"] = schedulesdf["portLatitude"].apply(lambda x: float(x))
schedulesdf["portLongitude"] = schedulesdf["portLongitude"].apply(lambda x: float(x))

vessels.csv

In [8]:
vehiclesdf = pd.read_csv("vessels.csv", sep="|")

vehiclesdf["CEU"] = vehiclesdf["CEU"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["DWT"] = vehiclesdf["DWT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["GT"] = vehiclesdf["GT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["NT"] = vehiclesdf["NT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["vesselType"] = vehiclesdf["vesselType"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["breadth"] = vehiclesdf["breadth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["depth"] = vehiclesdf["depth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["draft"] = vehiclesdf["draft"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["enginePower"] = vehiclesdf["enginePower"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["freshWater"] = vehiclesdf["freshWater"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["fuel"] = vehiclesdf["fuel"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["length"] = vehiclesdf["length"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxHeight"] = vehiclesdf["maxHeight"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxSpeed"] = vehiclesdf["maxSpeed"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxWidth"] = vehiclesdf["maxWidth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["rampCapacity"] = vehiclesdf["rampCapacity"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["yearBuilt"] = vehiclesdf["yearBuilt"].apply(lambda x: datetime.strptime(str(x), "%Y"))

Function for calculating time difference

In [9]:
def timeDiff(t1: datetime, t2: datetime, i="s"):
    diff = t1 - t2 if t1 >= t2  else t2 - t1
    if i == "s":
        return diff.total_seconds()
    elif i == "m":
        return diff.total_seconds() // 60
    elif i == "h":
        return diff.total_seconds() // 3600
    elif i == "d":
        return diff.days
    else:
        return 0

Adding WORKING_SIZE previous nodes to the ais_train dataframe

In [10]:
def prev_nodes_train(traindf: pd.DataFrame, WINDOW_SIZE=1):
    for j in range(1, WINDOW_SIZE + 1):
        traindf[f'latitude_{j}'] = traindf.groupby('vesselId')['latitude'].shift(j)
        traindf[f'longitude_{j}'] = traindf.groupby('vesselId')['longitude'].shift(j)
        traindf[f'cog_{j}'] = traindf.groupby('vesselId')['cog'].shift(j)
        traindf[f'sog_{j}'] = traindf.groupby('vesselId')['sog'].shift(j)
        
        traindf[f'time_{j}'] = traindf.groupby('vesselId')['time'].shift(j)
        traindf[f'time_{j}'] = (traindf['time'] - traindf[f'time_{j}']).dt.total_seconds()
    return traindf

def prev_nodes_test(traindf: pd.DataFrame, testdf: pd.DataFrame, WINDOW_SIZE=1):
    grouped_train = traindf.groupby('vesselId')
    
    for j in range(1, WINDOW_SIZE + 1):
        testdf[f'latitude_{j}'] = None
        testdf[f'longitude_{j}'] = None
        testdf[f'cog_{j}'] = None
        testdf[f'sog_{j}'] = None
        testdf[f'time_{j}'] = None

    for vesselId, test_vessel_data in testdf.groupby('vesselId'):
        if vesselId in grouped_train.groups:
            train_vessel_data = grouped_train.get_group(vesselId)
            for j in range(1, WINDOW_SIZE + 1):
                if len(train_vessel_data) >= j:
                    train_row = train_vessel_data.iloc[-j]
                    testdf.loc[test_vessel_data.index, f'latitude_{j}'] = train_row['latitude']
                    testdf.loc[test_vessel_data.index, f'longitude_{j}'] = train_row['longitude']
                    testdf.loc[test_vessel_data.index, f'cog_{j}'] = train_row['cog']
                    testdf.loc[test_vessel_data.index, f'sog_{j}'] = train_row['sog']
                    testdf.loc[test_vessel_data.index, f'time_{j}'] = (testdf.loc[test_vessel_data.index, 'time'] - train_row['time']).dt.total_seconds()
    return testdf

Checking if a boat is on a schedule

In [15]:
"""
def onSchedule(schedulesdf: pd.DataFrame, vesselId: str, time: datetime):
    if schedulesdf.index.name != 'vesselId':
        schedulesdf = schedulesdf.set_index('vesselId')
        
    if vesselId in schedulesdf.index:
        vessel_schedules = schedulesdf.loc[vesselId]
    else:
        return None
    
    fitting_schedules = (vessel_schedules["sailingDate"] >= time) & (vessel_schedules["arrivalDate"] <= time)
    if fitting_schedules.any():
        matching_schedules = vessel_schedules.loc[fitting_schedules]
        if len(matching_schedules) == 1:
            return matching_schedules.iloc[0]
    return None
"""
print(schedulesdf.columns)
if schedulesdf.index.name != 'vesselId':
    schedulesdf = schedulesdf.set_index('vesselId')

def onSchedule_optimized(testdf: pd.DataFrame, schedulesdf: pd.DataFrame, vesselId: str, time: datetime):
    if vesselId in schedulesdf.index:
        vessel_schedules = schedulesdf.loc[vesselId]
        if isinstance(vessel_schedules, pd.Series):
            vessel_schedules = vessel_schedules.to_frame().T
            
        fitting_schedules = (vessel_schedules["sailingDate"] >= time) & (vessel_schedules["arrivalDate"] <= time)
        if fitting_schedules.any():
            matching_schedules = vessel_schedules.loc[fitting_schedules]
            if len(matching_schedules) == 1:
                return matching_schedules.iloc[0]
    return None
print(schedulesdf.columns)
schedulesdf = schedulesdf.set_index('vesselId')
testdf["onSchedule"] = testdf.apply(lambda row: onSchedule_optimized(testdf, schedulesdf, row["vesselId"], row["time"]))
print(testdf.loc[:10])

Index(['shippingLineId', 'shippingLineName', 'arrivalDate', 'sailingDate',
       'portName', 'portId', 'portLatitude', 'portLongitude'],
      dtype='object')


KeyError: "None of ['vesselId'] are in the columns"

Finding Last Data Point

In [11]:
def lastData(testdf: pd.DataFrame, traindf: pd.DataFrame):
    vessels = testdf["vesselId"].unique().tolist()
    indexes = []

    for v in vessels:
        vessel_data = traindf[traindf['vesselId'] == v]
        indexes.append(vessel_data.sort_values(by="time").iloc[-1].name)

    return traindf.loc[indexes].sort_values(by="time")

#print(lastData(testdf, traindf).head())

Distance

In [12]:
def distance(lat1, long1, lat2, long2):
    return geodesic((lat1, long1), (lat2, long2)).meters

Checking if a vessel is near a port

In [13]:
from sklearn.neighbors import KDTree

PORT_DIST_THRESH = 3000

ports_coords = portsdf[["latitude", "longitude"]].values
kdtree = KDTree(ports_coords, leaf_size=30)

def nearPort(portsdf: pd.DataFrame, lat, long):
    rough_dist = PORT_DIST_THRESH / 100000
    indices = kdtree.query_radius([[lat, long]], r=rough_dist)
    for i in indices[0]:
        port = portsdf.iloc[i]
        if distance(lat, long ,port["latitude"],port["longitude"]) <= PORT_DIST_THRESH:
            return port
    return None

Checking if a vessel is far from land

In [14]:
MAP_LAND_PATH = "~/datasets/ne_10m_land.zip"
MAP_OCEAN_PATH = "~/datasets/ne_10m_ocean.zip"
    
land_world = gpd.read_file(MAP_LAND_PATH)
ocean_world = gpd.read_file(MAP_OCEAN_PATH)

def farFromCoast(latitude, longitude, land_world=land_world, thresh=12):
    land_world_projected = land_world.to_crs(epsg=3857)
    point = gpd.GeoSeries([Point(longitude, latitude)], crs="EPSG:4326").to_crs(epsg=3857)
    
    if land_world_projected.contains(point.iloc[0]).any():
        return False
    
    land_boundary = land_world_projected.geometry.boundary.unary_union
    distanceToCoast = point.distance(land_boundary)
    distanceNM = distanceToCoast.min() / 1852
    
    return distanceNM > thresh

Calculating average SOG

In [23]:
def avgSOG(traindf: pd.DataFrame):
    filtered_df = traindf[traindf["navstat"] == 0]
    sog_means = filtered_df.groupby("vesselId")["sog"].mean()
    return sog_means.to_dict()

avg = avgSOG(traindf)

Update Navstat for test data

In [31]:
def updateNavStat(traindf: pd.DataFrame, testdf: pd.DataFrame, schedulesdf: pd.DataFrame):
    statuses = ['Under_Way', 'Anchored', 'Moored']
    for status in statuses:
        testdf[status] = False

    grouped_train = traindf.groupby('vesselId')
    for vesselId, test_vessel_data in testdf.groupby('vesselId'):
        if vesselId in grouped_train.groups:
            last_train_row = grouped_train.get_group(vesselId).iloc[-1]
            for status in statuses:
                testdf.loc[test_vessel_data.index, status] = last_train_row[status]
                if last_train_row[status] is True:
                    testdf.loc[test_vessel_data.index, [s for s in statuses if s != status]] = False
        """for idx, row in test_vessel_data.iterrows():
            if onSchedule(schedulesdf, vesselId, row['time']) is not None:
                testdf.loc[idx, 'Moored'] = True
                testdf.loc[idx, 'Under_Way'] = False
                testdf.loc[idx, 'Anchored'] = False """
    return testdf

Creating Submission

In [16]:
def submission(sampledf: pd.DataFrame, lat_pred, lon_pred, deliver=False, name="SampleTest") -> pd.DataFrame:
    sample = sampledf.copy()

    for i in range(len(lat_pred)):
        sample.loc[i,"longitude_predicted"], sample.loc[i, "latitude_predicted"] = lon_pred[i], lat_pred[i]
    
    if deliver:
        sample.to_csv(name + ".csv", index=False)
        
    return sample

Copying Test and Train Dataframe

In [20]:
trainRFR = traindf.copy()
testRFR = testdf.copy()

Splitting data for ML

In [21]:
vessels = trainRFR["vesselId"].unique().tolist()
vessel_dict = {vessel: i for i, vessel in enumerate(vessels)}

ports = portsdf["portId"].unique().tolist()
port_dict = {port: i for i, port in enumerate(ports)}

# Adding previous nodes
window_size = 2
trainRFR = prev_nodes_train(trainRFR, window_size)
testRFR = prev_nodes_test(trainRFR, testRFR, window_size)

ML Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_result(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R^2 Score: {r2:.4f}')

y_lat = trainRFR["latitude"].copy()
y_lon = trainRFR["longitude"].copy()
X = trainRFR.drop(columns=['latitude', 'longitude'])

X = X.drop(columns=["etaRaw", "portId", "cog", "sog", "rot", "heading"])
#X["time"] = X["time"].astype("int64") / 10**9
X["time"] = pd.to_numeric(X["time"], errors='coerce')

# Splitting Navstat
X = pd.get_dummies(X, columns=["navstat"], prefix="", prefix_sep="")
X.rename(columns={"0": "Under_Way", "1": "Anchored", "5": "Moored"}, inplace=True)
X = X.drop(columns=["15"])

X["avgSOG"] = X["vesselId"].apply(lambda x: avg.get(x))
X["vesselId"] = X["vesselId"].map(vessel_dict)

X_train, X_test, y_train_lat, y_test_lat = train_test_split(X, y_lat, test_size=0.2)

clf_lat = RandomForestRegressor(n_estimators=10, random_state=42)
clf_lat.fit(X_train, y_train_lat)
y_pred_lat = clf_lat.predict(X_test)

print('Random Forest Model without Feature Engineering')
evaluate_result(y_test_lat, y_pred_lat)

#print(y_test_lat.tolist()[:10])
#print(y_pred_lat[:10])

X_train, X_test, y_train_lon, y_test_lon = train_test_split(X, y_lon, test_size=0.2)

#print(X_test.head())
clf_lon = RandomForestRegressor(n_estimators=10, random_state=42)
clf_lon.fit(X_train, y_train_lon)
y_pred_lon = clf_lon.predict(X_test)

print('Random Forest Model without Feature Engineering')
evaluate_result(y_test_lon, y_pred_lon)

#print(y_test_lon.tolist()[:10])
#print(y_pred_lon[:10])

#testRFR["nearPort"] = None
#testRFR["Under_Way"] = None
#testRFR["Anchored"] = None
#testRFR["Moored"] = None

test = testRFR.drop(columns=['ID', 'scaling_factor'])

test["vesselId"] = test["vesselId"].map(vessel_dict)
test = updateNavStat(X, test, schedulesdf)

test["avgSOG"] = test["vesselId"].apply(lambda x: avg.get(x))

#test["time"] = test["time"].astype("int64") / 10**9
test["time"] = pd.to_numeric(test["time"], errors='coerce')

cols = test.columns.tolist()
time_index = cols.index('time')
vesselId_index = cols.index('vesselId')
cols[time_index], cols[vesselId_index] = cols[vesselId_index], cols[time_index]
test = test[cols]

#print(testRFR.head())
#print(test.head())

lat_pred = clf_lat.predict(test)
lon_pred = clf_lon.predict(test)

#submission(sampledf, lat_pred, lon_pred, True, "Test1")
#print(lat_pred)
#print(lon_pred)

Random Forest Model without Feature Engineering
Mean Squared Error (MSE): 1.0404
Mean Absolute Error (MAE): 0.0596
R^2 Score: 0.9980
Random Forest Model without Feature Engineering
Mean Squared Error (MSE): 9.4084
Mean Absolute Error (MAE): 0.1177
R^2 Score: 0.9980


In [21]:
#submission(sampledf, lat_pred, lon_pred, True, "prediction")

,ID,longitude_predicted,latitude_predicted
0,0,-81.497905,31.146487
1,1,120.296297,14.816909
2,2,10.809007,38.335911
3,3,172.835428,-43.537958
4,4,-6.159701,48.550942
...,...,...,...
51734,51734,-31.899401,39.395808
51735,51735,-113.742467,27.983676
51736,51736,-113.830540,27.371876
51737,51737,17.450312,55.926400


In [37]:
trainCatB = traindf.copy()
testCatB = testdf.copy()

window_size = 2
trainCatB = prev_nodes_train(trainCatB, window_size)
testCatB = prev_nodes_test(trainCatB, testCatB, window_size)

In [27]:
"""
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_lat = trainCatB["latitude"].copy()
y_lon = trainCatB["longitude"].copy()

X = trainCatB.drop(columns=['latitude', 'longitude', 'etaRaw', 'portId', 'cog', 'sog', 'rot', 'heading'])

X["time"] = pd.to_numeric(X["time"], errors='coerce')

X = pd.get_dummies(X, columns=["navstat"], prefix="", prefix_sep="")
X.rename(columns={"0": "Under_Way", "1": "Anchored", "5": "Moored"}, inplace=True)
X = X.drop(columns=["15"])

X["avgSOG"] = X["vesselId"].apply(lambda x: avg.get(x))
X["vesselId"] = X["vesselId"].map(vessel_dict)

X_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_lat, test_size=0.2, random_state=42)
X_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_lon, test_size=0.2, random_state=42)

# Initialize CatBoost Regressor with default parameters
categorical_features = ['vesselId', 'Under_Way', 'Anchored', 'Moored']
catboost_lat = CatBoostRegressor(loss_function='RMSE', cat_features=categorical_features, verbose=100)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'iterations': [200, 500, 1000],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [3, 5, 7],
    'random_strength': [1, 2, 5]
}

# Hyperparameter tuning with GridSearchCV for latitude
grid_search_lat = GridSearchCV(estimator=catboost_lat, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_lat.fit(X_train_lat, y_train_lat)
best_params_lat = grid_search_lat.best_params_
print(f"Best parameters for latitude prediction: {best_params_lat}")

# Train the CatBoost model using the best parameters for latitude
catboost_lat_best = CatBoostRegressor(cat_features=categorical_features, **best_params_lat, verbose=100)
catboost_lat_best.fit(X_train_lat, y_train_lat)

# Hyperparameter tuning with GridSearchCV for longitude
catboost_lon = CatBoostRegressor(loss_function='RMSE', cat_features=categorical_features, verbose=100)
grid_search_lon = GridSearchCV(estimator=catboost_lon, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_lon.fit(X_train_lon, y_train_lon)
best_params_lon = grid_search_lon.best_params_
print(f"Best parameters for longitude prediction: {best_params_lon}")

# Train the CatBoost model using the best parameters for longitude
catboost_lon_best = CatBoostRegressor(cat_features=categorical_features, **best_params_lon, verbose=100)
catboost_lon_best.fit(X_train_lon, y_train_lon)

# Step 7: Evaluate the models
def evaluate_result(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R^2 Score: {r2:.4f}')

# Latitude predictions
y_pred_lat = catboost_lat_best.predict(X_test_lat)
print('CatBoost Latitude Prediction Results:')
evaluate_result(y_test_lat, y_pred_lat)

# Longitude predictions
y_pred_lon = catboost_lon_best.predict(X_test_lon)
print('CatBoost Longitude Prediction Results:')
evaluate_result(y_test_lon, y_pred_lon)

test = testCatB.drop(columns=['ID', 'scaling_factor'])

test["vesselId"] = test["vesselId"].map(vessel_dict)
test = updateNavStat(X, test, schedulesdf)

test["avgSOG"] = test["vesselId"].apply(lambda x: avg.get(x))
test["time"] = pd.to_numeric(test["time"], errors='coerce')

cols = test.columns.tolist()
time_index = cols.index('time')
vesselId_index = cols.index('vesselId')
cols[time_index], cols[vesselId_index] = cols[vesselId_index], cols[time_index]
test = test[cols]

# Step 8: Final predictions on test set and submission creation
lat_pred = catboost_lat_best.predict(test)
lon_pred = catboost_lon_best.predict(test)
"""

'\nfrom catboost import CatBoostRegressor\nfrom sklearn.model_selection import train_test_split, GridSearchCV\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\n\ny_lat = trainCatB["latitude"].copy()\ny_lon = trainCatB["longitude"].copy()\n\nX = trainCatB.drop(columns=[\'latitude\', \'longitude\', \'etaRaw\', \'portId\', \'cog\', \'sog\', \'rot\', \'heading\'])\n\nX["time"] = pd.to_numeric(X["time"], errors=\'coerce\')\n\nX = pd.get_dummies(X, columns=["navstat"], prefix="", prefix_sep="")\nX.rename(columns={"0": "Under_Way", "1": "Anchored", "5": "Moored"}, inplace=True)\nX = X.drop(columns=["15"])\n\nX["avgSOG"] = X["vesselId"].apply(lambda x: avg.get(x))\nX["vesselId"] = X["vesselId"].map(vessel_dict)\n\nX_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_lat, test_size=0.2, random_state=42)\nX_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_lon, test_size=0.2, random_state=42)\n\n# Initialize CatBoost Regresso

In [35]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_lat = trainCatB["latitude"].copy()
y_lon = trainCatB["longitude"].copy()

X = trainCatB.drop(columns=['latitude', 'longitude', 'etaRaw', 'portId', 'cog', 'sog', 'rot', 'heading'])

X["time"] = pd.to_numeric(X["time"], errors='coerce')

X = pd.get_dummies(X, columns=["navstat"], prefix="", prefix_sep="")
X.rename(columns={"0": "Under_Way", "1": "Anchored", "5": "Moored"}, inplace=True)
X = X.drop(columns=["15"])

X["avgSOG"] = X["vesselId"].apply(lambda x: avg.get(x))
X["vesselId"] = X["vesselId"].map(vessel_dict)

X_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_lat, test_size=0.2, random_state=42)
X_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_lon, test_size=0.2, random_state=42)

# Initialize CatBoost Regressor with default parameters
categorical_features = ['vesselId', 'Under_Way', 'Anchored', 'Moored']
catboost_lat = CatBoostRegressor(loss_function='RMSE', cat_features=categorical_features, verbose=100)

# Define parameter grid for hyperparameter tuning
param_distributions = {
    'iterations': [100, 200, 500],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [3, 5, 7],
    'random_strength': [1, 2, 5]
}

# Hyperparameter tuning with GridSearchCV for latitude
random_search_lat = RandomizedSearchCV(estimator=catboost_lat, param_distributions=param_distributions, n_iter=5, cv=2, n_jobs=-1, verbose=2, random_state=42)
random_search_lat.fit(X_train_lat, y_train_lat)
best_params_lat = random_search_lat.best_params_
print(f"Best parameters for latitude prediction: {best_params_lat}")

# Train the CatBoost model using the best parameters for latitude
catboost_lat_best = CatBoostRegressor(cat_features=categorical_features, **best_params_lat, verbose=100)
catboost_lat_best.fit(X_train_lat, y_train_lat)

# Hyperparameter tuning with GridSearchCV for longitude
catboost_lon = CatBoostRegressor(loss_function='RMSE', cat_features=categorical_features, verbose=100)
random_search_lon = RandomizedSearchCV(estimator=catboost_lon, param_distributions=param_distributions, n_iter=5, cv=2, n_jobs=-1, verbose=2, random_state=42)
random_search_lon.fit(X_train_lon, y_train_lon)
best_params_lon = random_search_lon.best_params_
print(f"Best parameters for longitude prediction: {best_params_lon}")

# Train the CatBoost model using the best parameters for longitude
catboost_lon_best = CatBoostRegressor(cat_features=categorical_features, **best_params_lon, verbose=100)
catboost_lon_best.fit(X_train_lon, y_train_lon)

# Step 7: Evaluate the models
def evaluate_result(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R^2 Score: {r2:.4f}')

# Latitude predictions
y_pred_lat = catboost_lat_best.predict(X_test_lat)
print('CatBoost Latitude Prediction Results:')
evaluate_result(y_test_lat, y_pred_lat)

# Longitude predictions
y_pred_lon = catboost_lon_best.predict(X_test_lon)
print('CatBoost Longitude Prediction Results:')
evaluate_result(y_test_lon, y_pred_lon)

test = testCatB.drop(columns=['ID', 'scaling_factor'])

test["vesselId"] = test["vesselId"].map(vessel_dict)
test = updateNavStat(X, test, schedulesdf)

test["avgSOG"] = test["vesselId"].apply(lambda x: avg.get(x))
test["time"] = pd.to_numeric(test["time"], errors='coerce')

cols = test.columns.tolist()
time_index = cols.index('time')
vesselId_index = cols.index('vesselId')
cols[time_index], cols[vesselId_index] = cols[vesselId_index], cols[time_index]
test = test[cols]

# Step 8: Final predictions on test set and submission creation
lat_pred = catboost_lat_best.predict(test)
lon_pred = catboost_lon_best.predict(test)

Fitting 2 folds for each of 5 candidates, totalling 10 fits
0:	learn: 20.7218494	total: 117ms	remaining: 11.6s
99:	learn: 1.3956507	total: 17.9s	remaining: 0us
[CV] END depth=4, iterations=100, l2_leaf_reg=7, learning_rate=0.1, random_strength=1; total time=  32.0s
0:	learn: 20.7637504	total: 330ms	remaining: 2m 44s
100:	learn: 1.3106280	total: 34.6s	remaining: 2m 16s
200:	learn: 1.0831108	total: 1m 8s	remaining: 1m 41s
300:	learn: 0.9745827	total: 1m 41s	remaining: 1m 6s
400:	learn: 0.8994724	total: 2m 14s	remaining: 33.1s
499:	learn: 0.8406887	total: 2m 46s	remaining: 0us
[CV] END depth=6, iterations=500, l2_leaf_reg=5, learning_rate=0.1, random_strength=5; total time= 3.1min


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0:	learn: 20.7216246	total: 193ms	remaining: 19.2s
99:	learn: 1.3875740	total: 17.6s	remaining: 0us
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1, random_strength=1; total time=  31.8s
0:	learn: 20.7641282	total: 417ms	remaining: 3m 28s
100:	learn: 1.2796485	total: 34.2s	remaining: 2m 15s
200:	learn: 1.0735237	total: 1m 8s	remaining: 1m 41s
300:	learn: 0.9722561	total: 1m 41s	remaining: 1m 7s
400:	learn: 0.9081901	total: 2m 15s	remaining: 33.5s
499:	learn: 0.8537098	total: 2m 47s	remaining: 0us
[CV] END depth=6, iterations=500, l2_leaf_reg=5, learning_rate=0.1, random_strength=5; total time= 3.1min
0:	learn: 20.7202494	total: 233ms	remaining: 23.1s
99:	learn: 1.3779422	total: 17.9s	remaining: 0us
[CV] END depth=4, iterations=100, l2_leaf_reg=7, learning_rate=0.1, random_strength=1; total time=  32.1s
0:	learn: 20.7070827	total: 398ms	remaining: 3m 18s
100:	learn: 1.1203228	total: 46.1s	remaining: 3m 2s
200:	learn: 0.9254617	total: 1m 31s	remaining: 2m 15s
300:	lear

In [36]:
submission(sampledf, lat_pred, lon_pred, True, "catboost_prediction_3")

,ID,longitude_predicted,latitude_predicted
0,0,-80.618599,31.485052
1,1,120.428808,14.996091
2,2,10.884976,38.740143
3,3,172.958424,-42.062873
4,4,-6.006832,49.086967
...,...,...,...
51734,51734,-49.098934,32.982273
51735,51735,-67.317577,29.405737
51736,51736,-137.460310,26.761694
51737,51737,10.961628,51.858896


0:	learn: 65.3951342	total: 586ms	remaining: 4m 52s
100:	learn: 3.4421365	total: 39.6s	remaining: 2m 36s
200:	learn: 2.8676606	total: 1m 1s	remaining: 1m 31s
300:	learn: 2.5646921	total: 1m 23s	remaining: 55.1s
400:	learn: 2.3482048	total: 1m 44s	remaining: 25.8s
499:	learn: 2.1587949	total: 2m 4s	remaining: 0us
[CV] END depth=8, iterations=500, l2_leaf_reg=7, learning_rate=0.05, random_strength=2; total time= 2.4min
